# Computational Theory Assessment

In [1]:
import numpy as np

np.seterr(over='ignore') # Error in problem 4 due to large exponentials

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## <Strong>Problem 1 </strong>: Binary Words and Operations

> **AI Assistance Disclosure**  
> The test code block structures (for `Parity`, `Choose`, `Maj`, and all Σ/σ functions) were **AI-generated using ChatGPT (2025)** to maintain consistent formatting and binary visualisation style across functions.  
> All logic and final verification were completed manually.


### 1. Parity 
**Parity** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Parity}(x, y, z) = x \oplus y \oplus z
$$

where ⊕ shows the **bitwise XOR** operation.

For each bit position, the result is 1 if an **odd number** of x, y and z contain a 1 in that position, otherwise 0.  
Parity acts as an **odd-bit detector**, ensuring that even a single bit change in the inputs alters the output.

**Simply Said:** Do I have an odd number of 1s(INPUT 1) or an even number of 1s(INPUT 0)?

**Why XOR is used**
- **XOR** stands for *exclusive OR*. 
- It's a logical operation that compares two bits and outputs **1 if they are different** and **0 if they are the same**.
- XOR is **fast**, **branch-free**, and supported directly in Hardware.  
- It implements addition mod 2 at the bit level.  

**Alternative forms in research**

Some implementations describe *Parity* differently:

| Formulation | Description | Why I Didn't Use|
| :-- | :-- | :-- |
| `(x + y + z) % 2` | Modulo-2 sum - Simplistic show of odd/even nature by summing bits and taking the remainder mod 2. |  You’d need to apply to each bit - expensive for 32-bit words. |
| `bin(x ^ y ^ z).count("1") % 2`  | Bit Count Method - parity of all bits within one number| Very slow, string conversion, not bitwise.|


**Truth Table (for 1-bit inputs)**

| x | y | z | Parity |
|:--:|:--:|:--:|:--:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

In [2]:
# Uses np.uint32() : https://numpy.org/devdocs/user/basics.types.html
def parity(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the parity (XOR) of three 32-bit numbers.

    Each bit of the result is 1 if an odd number of the inputs
    have a 1 in that position, otherwise 0.

    Parameters:
        x (uint32): First 32-bit integer.
        y (uint32): Second 32-bit integer.
        z (uint32): Third 32-bit integer.

    Returns:
        np.uint32: The XOR (parity) of the three input values.
    """
    return np.uint32(x ^ y ^ z)


In [3]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("              x =", np.binary_repr(x, 32))
print("              y =", np.binary_repr(y, 32))
print("              z =", np.binary_repr(z, 32))
 
# Perform Parity
result = parity(x, y, z)

# Show result in binary
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Parity(x, y, z) =", np.binary_repr(result, 32))

              x = 00000000000000000000000010101010
              y = 00000000000000000000000011001100
              z = 00000000000000000000000001111000
Parity(x, y, z) = 00000000000000000000000000011110


### 2. Choose(x, y, z)

**Choose** is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

Where:  
- `∧` is the **bitwise AND** operation,  
- `⊕` is the **bitwise XOR**,  
- `¬` is the **bitwise NOT** (flips every bit).  

**Objective**  
Choose uses `x` as a **bit-selector** - for every bit position:  
- if bit from x = 1, take the corresponding bit from y;  
- if bit from x = 0, take it from z.  

So x acts like a 32-bit “key” that chooses between y and z.


**Simply said:** **Choose** picks bits from `y` or `z` depending on whether `x`’s bit is 1 or 0.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (~x & z)` | OR instead of XOR | Non-Uniformity within problem set  |
| `z ^ (x & (y ^ z))` | Algebraic | Doesn't Match Spec, Not Great Readability |

**Why this method (`np.uint32((x & y) ^ (~x & z))`)?**  
- It exactly follows the standard equation.  
- **np.uint32** confines results to 32 bits (no overflow).  
- Bitwise operations are hardware-level fast and branch-free — each bit is processed independently.  


**Example (1-bit truth table)**  

| x | y | z | Choose(x,y,z) |
|:-:|:-:|:-:|:-------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [4]:
def choose(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Return the result of choosing bits from y and z based on x.
    
    For each bit position, if the bit in x is 1, take the bit from y,
    otherwise take the bit from z.

    bitwise NOT operator (~) flips each bit: 0 becomes 1, and 1 becomes 0.
    bitwise AND operator (&) compares each bit of two numbers and returns 1 if both bits are 1, otherwise returns 0.
    bitwise XOR operator (^) compares each bit of two numbers and returns 1 if the bits are different, otherwise returns 0.

    Parameters:
        x (uint32): The mask.
        y (uint32): The first number.
        z (uint32): The second number.
    """
    return np.uint32((x & y) ^ (~x & z))


In [5]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("             x  =", np.binary_repr(x, 32))
print("             y  =", np.binary_repr(y, 32))
print("             z  =", np.binary_repr(z, 32))

# Perform Choose
result = choose(x, y, z)

# Show result in binary
print("Choose(x, y, z) =", np.binary_repr(result, 32))


             x  = 00000000000000000000000010101010
             y  = 00000000000000000000000011001100
             z  = 00000000000000000000000001111000
Choose(x, y, z) = 00000000000000000000000011011000


### 3. Maj(x, y, z)

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\text{Maj}(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)
$$


**Objective**  
The **Majority** function outputs 1 for each bit position where **two or more**
of the corresponding bits in `x`, `y`, and `z` are 1.
This makes it act as a “bitwise decider,” combining information from multiple inputs
to strengthen the *diffusion* of bits.


**Simply said:**  Majority - whichever value (0 or 1) occurs most often across x, y, and z.

**Alternative forms and interpretations**

| Formulation | Description | Why I Didn't Use|
|:--|:--|:--|
| `(x & y) \| (x & z) \| (y & z)` | Equivalent with OR instead of XOR | Less conventional - XOR keeps consistency with other SHA logical functions. |
| `(x & y) \| (z & (x \| y))` | Boolean simplification | Harder to read - hides the “majority” meaning behind extra grouping. |
| `((x ^ y) & z) \| (x & y)` | Optimized algebraic rearrangement |  Used for performance in C, unclear for teaching purposes & no scalability |


**Why this method (`np.uint32((x & y) ^ (x & z) ^ (y & z))`)?**  
- It directly mirrors the **official FIPS definition**, ensuring correctness and traceability.  
- Using `np.uint32` confines operations to 32-bit words, matching SHA-256’s internal word size.  
- The XOR version maintains consistency with the rest of SHA’s logic functions, which are all expressed with XOR, AND, and NOT.  
- It’s branch-free, efficient, and clear.

**Example (1-bit inputs)**  

| x | y | z | Maj(x,y,z) |
|:-:|:-:|:-:|:-----------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |


In [6]:
def Maj(x: np.uint32, y: np.uint32, z: np.uint32):
    """
    Compute the majority function.

    Maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)

    For each bit position, the output is 1 if two or more of
    the corresponding bits in x, y, and z are 1, otherwise 0.

    Parameters
    ----------
    x, y, z : np.uint32
        32-bit unsigned integers.

    Returns
    -------
    np.uint32
        The bitwise majority of x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)

In [7]:
# Initiate Test values
x = np.uint32(0b10101010)
y = np.uint32(0b11001100)
z = np.uint32(0b01111000)

# Show original inputs
print("           x =", np.binary_repr(x, 32))
print("           y =", np.binary_repr(y, 32))
print("           z =", np.binary_repr(z, 32))

# Perform Majority
result = Maj(x, y, z)

# Show result in binary
print("Maj(x, y, z) =", np.binary_repr(result, 32))


           x = 00000000000000000000000010101010
           y = 00000000000000000000000011001100
           z = 00000000000000000000000001111000
Maj(x, y, z) = 00000000000000000000000011101000


#### Helper Functions 
- I chose to use **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and       efficient implementation.  This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency,  and makes the code easier to test and understand.

In [8]:
# Helper functions for Sigma functions
# Rotate-right operation for 32-bit words - Helper function for Sigma functions
def ROTR(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right operation for 32-bit words."""
    return np.uint32((x >> n) | (x << np.uint32(32 - n)))

# Shift-right operation for 32-bit words - Helper function for Sigma functions
def SHR(x: np.uint32, n: int) -> np.uint32:
    """Shift-right operation for 32-bit words."""
    return np.uint32(x >> n)


### 4. Sigma Zero - Σ₀(x) 

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_0^{\{256\}}(x) = \text{ROTR}^2(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)
$$

**Objective**  
The **Σ₀** function performs three right-rotations of the 32-bit `x`
by 2, 13, and 22 bits, then XORs the results.

This process ensures that bits from different positions in `x`
influence each other in later stages of the SHA-256 computation,
creating strong *diffusion* — a key cryptographic property
where small changes in input cause large, unpredictable changes in output.


**Simply Said:**  Σ₀(x) “scrambles” the bits of x by spinning them around and combining the three versions with XOR to mix them thoroughly.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `np.right_shift` with masking | Implements shifts manually | More verbose and less readable than `ROTR()` helper. |

**Why I Chose This Method**  
- The XOR-based rotation exactly matches the FIPS specification.  
- Using the shared `ROTR()` helper keeps all rotations consistent and clear across Σ₀, Σ₁, σ₀, and σ₁.  
- NumPy’s `np.uint32` enforces 32-bit wrapping automatically, avoiding overflow or sign issues.  
- It’s fast & readable.


**Behaviour Summary**
| Operation | Description |
|------------|-------------|
| ROTR²(x)  | Circular right rotation by 2 bits |
| ROTR¹³(x) | Circular right rotation by 13 bits |
| ROTR²²(x) | Circular right rotation by 22 bits |

The final result is the XOR (⊕) of these three rotated words.


In [9]:
def Sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₀ (Sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    np.uint32
        The resulting 32-bit word after applying Σ₀(x).
    """
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)


In [10]:
# Initiate Test value
x = np.uint32(0b10110010)


# Display result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Original Number:", np.binary_repr(x, 32))
print("Sigma0 Result  :", np.binary_repr(Sigma0(x), 32))


Original Number: 00000000000000000000000010110010
Sigma0 Result  : 10000101100100101100100000101100


### 5. Sigma One - Σ₁(x) 

Defined in the [Secure Hash Standard (FIPS 180-4, §4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\Sigma_1^{\{256\}}(x) = \text{ROTR}^6(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)
$$


**Objective**  
The **Σ₁** function performs three right rotations of the 32-bit word `x`
by 6, 11, and 25 bits, then XORs the results.
This operation spreads information across bits and rounds, ensuring
that small differences in the input word produce large, unpredictable
differences in the output. It enhances *diffusion* and *non-linearity*,
both essential for cryptographic strength in SHA-256.

**Simply Said:** Σ₁(x) twists the bits of `x` three times by different amounts, then combines them.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `(x >> 6) ^ (x >> 11) ^ (x >> 25)` | Simple shifts instead of rotations | Loses information — bits are discarded instead of wrapped around. |
| `(ROTR(x,6) + ROTR(x,11) + ROTR(x,25)) & 0xFFFFFFFF` | Integer addition, not XOR | Incorrect — adds values numerically instead of mixing bits logically. |

---

**Why I Chose This Method**  
- The XOR-based rotation exactly matches the FIPS specification.  
- Using the shared `ROTR()` helper keeps all rotations consistent and clear across Σ₀, Σ₁, σ₀, and σ₁.  
- NumPy’s `np.uint32` enforces 32-bit wrapping automatically, avoiding overflow or sign issues.  
- It’s fast & readable.

**Behaviour Summary**
| Operation | Description |
|------------|--------------|
| ROTR⁶(x)  | Circular right rotation by 6 bits |
| ROTR¹¹(x) | Circular right rotation by 11 bits |
| ROTR²⁵(x) | Circular right rotation by 25 bits |

The final output is the XOR (⊕) of these three rotated words.


In [11]:
def Sigma1(x: np.uint32) -> np.uint32:
    """
    Compute the Σ₁ (Sigma one) function.

    This function performs three right-rotations of a 32-bit word `x` by
    6, 11, and 25 bits, and returns the bitwise XOR of these results:

        Σ₁(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.

    Returns
    np.uint32
        The resulting 32-bit word after applying Σ₁(x).
    """
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)


In [12]:
# Initiate Test value
x = np.uint32(0b10110010)

# Show original input
print("x     =", np.binary_repr(x, 32))

# Perform Σ₁(x)
result = Sigma1(x)



# Display the result in binary for readability
# Uses np.binary_repr(): https://numpy.org/doc/2.1/reference/generated/numpy.binary_repr.html
print("Σ₁(x) =", np.binary_repr(Sigma1(x), 32))


x     = 00000000000000000000000010110010
Σ₁(x) = 11011110010000000101100100000010


### 6. Small Sigma Zero - σ₀(x) 

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\sigma_0^{\{256\}}(x) = \text{ROTR}^7(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^3(x)
$$


**Objective**  
σ₀(x) is one of the *message schedule* functions used in SHA-256.  
It takes a 32-bit x, rotates it right by 7 and 18 bits, then performs a simple right shift by 3 bits (filling with zeros).  

The three results are XORed together to create a new, intensely mixed version of `x`.  
This helps improving **diffusion** (the spreading of something more widely).

**Simply Said:**  σ₀(x) spins 'x' around twice and gives it one shove to the right. The result is then XOR'd : XOR (exclusive OR) outputs 1 when bits differ, and 0 when they’re the same.


**Alternative forms and interpretations**

| Formulation | Description | Why I Didn’t Use |
|:--|:--|:--|
| `(x >> 7) ^ (x >> 18) ^ (x >> 3)` | All logical shifts | Loses information - shifts discard bits instead of rotating them. |
| `np.roll()` on bit arrays | Simulates rotation on arrays | Slower  |


**Why I Chose This Method**  
- Follows the exact equation defined in FIPS 180-4 for SHA-256.   
- I chose to use **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and       efficient implementation.  This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency,  and makes the code easier to test and understand.
- NumPy’s `np.uint32` keeps all arithmetic confined to 32 bits, matching the real hardware behaviour.  
- It’s simple & readable.


**Example (8-bit demo)**  
(Shown shorter for readability — real SHA-256 uses 32 bits.)

| Step | Operation | Result |
|:--|:--|:--|
| Input | x = `10010011` |  |
| 1 | ROTR⁷(x) | `11001001` |
| 2 | ROTR¹⁸(x) | `01100100` |
| 3 | SHR³(x) | `00010010` |
| XOR all | σ₀(x) | `10111111` |

The final output is the XOR (⊕) of these three results.


In [13]:
def sigma0(x: np.uint32) -> np.uint32:
    """
    Compute the σ₀ (sigma zero) function used in SHA-256.

    Defined in FIPS 180-4 (Section 4.1.2) as:
        σ₀(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)

    This function performs two right-rotations followed by one right-shift,
    then XORS the results together.

    Parameters
    x : np.uint32
        32-bit unsigned integer input word.
   
    Returns
    np.uint32
        The resulting 32-bit word after applying σ₀(x).
    """

    return np.bitwise_xor(np.bitwise_xor(ROTR(x, 7), ROTR(x, 18)), SHR(x, 3))

In [14]:
#Testing
# Initiate Test value
x = np.uint32(0b10110010)

# Show original input
print("x     =", np.binary_repr(x, 32))

# Perform σ₀(x)
result = sigma0(x)

# Show result in binary
print("σ₀(x) =", np.binary_repr(result, 32))


x     = 00000000000000000000000010110010
σ₀(x) = 01100100001011001000000000010111


### σ₁(x) — sigma One

Defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.2)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

$$
\sigma_1^{\{256\}}(x) = \operatorname{ROTR}^{17}(x)\ \oplus\ \operatorname{ROTR}^{19}(x)\ \oplus\ \operatorname{SHR}^{10}(x)
$$

**Objective**  
Performs two **bitwise right rotations** and one **logical right shift** on a 32-bit word by 17, 19, and 10 bits respectively, then XORs the results together.  
This function contributes to **diffusion** in the **SHA-256 message schedule**, expanding message words so that every bit influences later rounds.

**Simply Said:**  
σ₁(x) spins `x` twice, slides it once to the right, and then combines them using **XOR** — where XOR (exclusive OR) outputs 1 only when the bits differ.

**Alternative forms and interpretations**  
- Sometimes written as  
  $$
  \sigma_1(x) = (x \ggg 17)\ \oplus\ (x \ggg 19)\ \oplus\ (x \gg 10)
  $$  
  where $(\ggg)$ means *rotate right* (ROTR) and $(\gg)$ means *logical shift right* (SHR).  
- The difference between rotate and shift: **rotate** wraps bits around, while **shift** discards bits that move past the edge and fills with zeros.

**Why I chose this method**  
I used **helper functions** (`ROTR` and `SHR`) for a clean, encapsulated, and efficient implementation.  
This keeps these operations reusable across all of the Σ and σ functions, ensures 32-bit consistency, and makes the code easier to test and understand.

**Example (8-bit demo)**  
(Shown shorter for readability — real SHA-256 uses 32 bits.)

| Step | Operation | Result |
|:--|:--|:--|
| Input | x = 10110010 |  |
| 1 | ROTR¹⁷(x) | 01011001 |
| 2 | ROTR¹⁹(x) | 10010110 |
| 3 | SHR¹⁰(x) | 00001011 |
| XOR all | σ₁(x) | 11010100 |

The final output is the XOR (⊕) of these three results, demonstrating how σ₁(x) mixes the rotated and shifted versions of `x` to achieve diffusion.


In [15]:
def sigma1(x: np.uint32) -> np.uint32:
    """
    σ₁(x) — Lowercase sigma one 

    Computes: ROTRⁱ⁷(x) XOR ROTR¹⁹(x) XOR SHR¹⁰(x)

    Parameters
    x : np.uint32
        32-bit input word.

    Returns
    np.uint32
        32-bit result of σ₁(x).

    """
    return ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10)


In [16]:
# Initiate Test value
x = np.uint32(0b10110010)

# Show original input
print("x     =", np.binary_repr(x, 32))

# Perform σ₁(x)
result = sigma1(x)

# Show result in binary
print("σ₁(x) =", np.binary_repr(result, 32))


x     = 00000000000000000000000010110010
σ₁(x) = 00000000010011110100000000000000


## <strong>Problem 2 </strong>: Fractional Parts of Cube Roots

### <strong>Prime Number</strong> Function

#### Definition
A **prime number** is a positive integer greater than 1 that has no positive factors other than 1 and itself
([Khan Academy, 2022](https://www.khanacademy.org/math/arithmetic/factors-multiples/prime-numbers/v/recognizing-prime-numbers)).


For example, `2`, `3`, `5`, and `7` are prime numbers, while  `8` is not because it has multiple divisors (2 & 4).

The `primes(n)` function I created generates the **first _n_ prime numbers**, starting from 2.  
It uses **trial division**, where each candidate number is tested for divisibility by every integer up to its square root.  
This approach is conceptually simple and more than efficient enough for small values like the first 64 primes used in SHA-256.

---

#### Other Implementations
There are several alternative ways to check or generate prime numbers in Python:

| Formulation | Description | Why I Didn’t Use |
| :-- | :-- | :-- |
| **Sieve of Eratosthenes** | Efficient algorithm that marks multiples of primes in a range, producing all primes up to a limit. | Adds complexity when only 64 primes are needed.  ([GeeksforGeeks, 2022](https://www.geeksforgeeks.org/python/python-program-for-sieve-of-eratosthenes/)). |
| **Built-in Libraries (`sympy.primerange`)** | Uses external packages that can instantly list primes. | External libraries aren’t permitted for this assessment to my knowledge. ([SymPy Documentation](https://docs.sympy.org/latest/modules/ntheory.html)). |
| **Optimized Trial Division** | Same logic as mine but skips even numbers above 2 and divides only by previously found primes. | Faster but less readable. |

---

#### Why I Chose This Method
I chose the **basic trial division** method because:
- It uses only **NumPy**.  
- It’s **simple and readable**, making it easy to explain and test.  
- It quickly handles up to **n = 64**, which is all we need for the SHA-256 constants.  
- It clearly demonstrates the logic behind prime checking.

It’s not the fastest, but it’s the clearest!

---

#### Run Through of Method
1. Start with an empty list `primes_list` and set the first candidate number to 2.  
2. For each candidate, assume it’s prime (`is_prime = True`).  
3. Loop through all numbers from `2` to the **square root** of the candidate (`np.sqrt(candidate)`):  
   - If any of those divides evenly, mark `is_prime = False`.  
4. If still prime, append the candidate to the list.  
5. Increase the candidate by 1 and repeat until we have `n` primes.  
6. Return the list of primes.


In [17]:
# Step One: Generate first n prime numbers
def primes(n):
    """
    Return the first n prime numbers using simple trial division.

    Parameters:
        n (int): The number of prime numbers to generate.
    """
    primes_list = []
    candidate = 2

    while len(primes_list) < n:
        is_prime = True
        for i in range(2, int(np.sqrt(candidate)) + 1):
            if candidate % i == 0:
                is_prime = False
                break
        if is_prime:
            primes_list.append(candidate)
        candidate += 1

    return primes_list

# Example : Test to see first 64 primes are correct
print(primes(64))  

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]


### <strong>Cube Root</strong> Calculation

#### Definition
**Cube roots** are the inverse operation of cubing a number, meaning they help us find which number, when multiplied by itself three times, gives the original number ([GeeksforGeeks, 2023](https://www.geeksforgeeks.org/maths/how-to-teach-cube-roots/)). 

For example, the cube root of 8 is 2 because $$ 2^3 = 8 $$  
In this problem, the cube roots of the first 64 prime numbers are used to generate constants for the SHA-256 algorithm - constants in SHA-256 are fixed numbers built into the algorithm.
They are used in the inner mathematical loops of the hash to mix bits together!

Each cube root gives a number like **1.259921...**, and it’s the **fractional part** (the numbers after the decimal) that we’ll use later to form the constants.

---

#### Other Implementations

| Method | Description | Why I Didn’t Use |
| :-- | :-- | :-- |
| **Newton–Raphson Method** | An iterative algorithm that refines an estimate of the cube root using the formula $$ x_{k+1} = \frac{2x_k + n/x_k^2}{3} $$ | Accurate but more complex & NumPy already provides a precise function. |

 - Reference: [Flexiple – Newton–Raphson Method in Python](https://flexiple.com/python/newton-raphson-method-python)
---

#### Why I Chose This Method
I used **NumPy’s `np.cbrt()`** function because it:
- Is **numerically stable**.  
- Handles both positive and negative inputs correctly.  
- Accurate results.  
- Clean and readable code for this small dataset.
([NumPy Documentation – `numpy.cbrt`](https://numpy.org/doc/stable/reference/generated/numpy.cbrt.html)).
---

#### Run Through of Method
1. Generate the first 64 prime numbers using the `primes(n)` function.  
2. Pass the list of primes into NumPy’s cube root function:
   ```python
   cube_roots = np.cbrt(prime_list)
   ```
   
3. Print first 10.


In [18]:
# Step Two: Compute cube roots of first 64 primes
prime_list = primes(64)
cube_roots = np.cbrt(prime_list)

# Display first few results
for i, val in enumerate(cube_roots[:10]):
    print(f"Prime {prime_list[i]:>3}  : Cube root = {val:.15f}")


Prime   2  : Cube root = 1.259921049894873
Prime   3  : Cube root = 1.442249570307408
Prime   5  : Cube root = 1.709975946676697
Prime   7  : Cube root = 1.912931182772389
Prime  11  : Cube root = 2.223980090569316
Prime  13  : Cube root = 2.351334687720758
Prime  17  : Cube root = 2.571281590658235
Prime  19  : Cube root = 2.668401648721945
Prime  23  : Cube root = 2.843866979851565
Prime  29  : Cube root = 3.072316825685847


### <strong>Fractional Parts of Cube Roots</strong>

#### Definition
The **fractional part** of a number is the part after the decimal point.  
For example, the fractional part of `3.1415` is ``0.1415``.  
The fractional parts of the cube roots of the first 64 primes are extracted, these will then be used to generate the 32-bit constants for SHA-256.

Equation:
$$
\text{fractional part} = x - \lfloor x \rfloor
$$

([MathWorld – Fractional Part Function](https://mathworld.wolfram.com/FractionalPart.html))

---

#### Why This Step Is Needed
The fractional parts ensure that the constants are **non-repeating & uniformly distributed**,  
providing randomness without relying on arbitrary values. 

According to the **Secure Hash Standard**  
([NIST FIPS 180-4, Section 4.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)),  
these constants are derived from the fractional parts of the cube roots of prime numbers.  

This guarantees that the constants are *predictable yet unbiased*

---

#### Run Through of Method
1. Compute cube roots of the first 64 primes.  
2. Extract the fractional part of each cube root:
   ```python
   fractional_parts = cube_roots % 1
   ```


In [19]:
# Step Three: Extract fractional parts of cube roots
fractional_parts = cube_roots % 1 

# Display first few fractional parts
for i in range(10):
    print(f"Prime {prime_list[i]:>3} : Fractional Part = {fractional_parts[i]:.15f}")


Prime   2 : Fractional Part = 0.259921049894873
Prime   3 : Fractional Part = 0.442249570307408
Prime   5 : Fractional Part = 0.709975946676697
Prime   7 : Fractional Part = 0.912931182772389
Prime  11 : Fractional Part = 0.223980090569316
Prime  13 : Fractional Part = 0.351334687720758
Prime  17 : Fractional Part = 0.571281590658235
Prime  19 : Fractional Part = 0.668401648721945
Prime  23 : Fractional Part = 0.843866979851565
Prime  29 : Fractional Part = 0.072316825685847


### <strong>Creating 32-bit Constants</strong>

#### Overview
This final step converts the fractional parts of the cube roots into **32-bit integer constants**.  
Each fractional value is multiplied by $$ 2^{32} $$ to extract the first 32 binary digits.  
NumPy’s `np.floor()` function truncates (not rounds) the result, keeping only the integer portion.  
The array is then cast to `np.uint32` to match the 32-bit word size used in the **SHA-256 algorithm**.

#### Function: `creatingConstants()`
This function:
1. Multiplies each fractional part by $$ 2^{32} $$  
2. Uses `np.floor()` to drop the fractional component.  
3. Converts the values into unsigned 32-bit integers.  
4. Prints the first ten constants in both decimal and hexadecimal formats for verification.  
5. Returns all 64 constants as a NumPy array.

These constants form the **K₀ – K₆₃ table** defined in the Secure Hash Standard and  
are used during each of the 64 rounds of SHA-256.

#### References
- National Institute of Standards and Technology (2015).  
  *Secure Hash Standard (SHS)*, FIPS PUB 180-4.  
  [https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)  
- NumPy Documentation. *numpy.floor — Return the floor of the input*  
  [https://numpy.org/doc/stable/reference/generated/numpy.floor.html](https://numpy.org/doc/stable/reference/generated/numpy.floor.html)



In [20]:
def creatingConstants(fractional_parts, primes):
    """
    Step Four: Convert fractional parts to 32-bit integer constants.

    Each fractional part is multiplied by 2**32 to extract its first 32 binary
    digits. The `np.floor()` function drops everything after the decimal point,
    so only the integer portion remains. The final array is cast to `np.uint32`
    to match the 32-bit word size used in SHA-256.

    Parameters
    ----------
    fractional_parts : numpy.ndarray
        The fractional parts of the cube roots of the first 64 prime numbers.
    primes : list
        The list of prime numbers - for demonstrating.

    Returns
    -------
    numpy.ndarray
        Array of unsigned 32-bit integers representing the SHA-256 constants.
    """
    constants = np.floor(fractional_parts * (2**32)).astype(np.uint32)

    # Display first few results for verification
    print("Prime   Constant (decimal)   Constant (hex)")
    print("--------------------------------------------")
    for i in range(10):
        dec_val = constants[i]
        hex_val = f"{dec_val:08x}"
        print(f"{primes[i]:>5}   {dec_val:>15}         {hex_val}")

    return constants


constants = creatingConstants(fractional_parts, prime_list)



Prime   Constant (decimal)   Constant (hex)
--------------------------------------------
    2        1116352408         428a2f98
    3        1899447441         71374491
    5        3049323471         b5c0fbcf
    7        3921009573         e9b5dba5
   11         961987163         3956c25b
   13        1508970993         59f111f1
   17        2453635748         923f82a4
   19        2870763221         ab1c5ed5
   23        3624381080         d807aa98
   29         310598401         12835b01


### <strong>Comparison with Official SHA-256 Constants</strong>

To confirm the correctness of the generated constants, the first eight values are compared  
against the official SHA-256 constants listed in the Secure Hash Standard (FIPS 180-4, page 11).  

Each comparison checks if the hexadecimal constant from the last few cells 
matches the corresponding published value from NIST.

A result of `: True` means the generated constant is identical to the official one,  
confirms the implementation is correct.


In [21]:
# Compare with official SHA-256 constants from FIPS 180-4
official_hex = [
    "428a2f98", "71374491", "b5c0fbcf", "e9b5dba5",
    "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5"
]

for i in range(8):
    mine = f"{constants[i]:08x}"
    print(f"{i}: {mine} == {official_hex[i]} : {mine == official_hex[i]}")


0: 428a2f98 == 428a2f98 : True
1: 71374491 == 71374491 : True
2: b5c0fbcf == b5c0fbcf : True
3: e9b5dba5 == e9b5dba5 : True
4: 3956c25b == 3956c25b : True
5: 59f111f1 == 59f111f1 : True
6: 923f82a4 == 923f82a4 : True
7: ab1c5ed5 == ab1c5ed5 : True


## <strong>Problem 3 </strong>: Padding


Before a message can be processed by the SHA-256 algorithm, it must be **padded** so that its total length is a multiple of **512 bits (64 bytes)**.  This is to prepare the data before pushing it to the hashing algorithm.

This ensures the algorithm can handle *any* message size in fixed-sized chunks.
 
**Padding** process as defined in the [Secure Hash Standard (FIPS 180-4, § 5.1.1, 5.2.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) works as follows:  

1. Start with the message of length (L) bits.  
2. Add a single `1` bit (`0x80` in hex, that marks the end of the original data.  
3. Add enough `0` bits so that the total length ≡ 448 (mod 512).  
   – This leaves 64 bits (8 bytes) at the end to note the message length.  
4. Add the **64-bit big-endian** representation of L (the original bit length).  
   – "Big-endian" means the most significant byte is written first. 
5. The result is one or more **512-bit blocks**, ready for hashing.

These steps guarantee every padded message is exactly 512-bits. 


### Simply Said
Padding makes sure every message fits perfectly into 512-bit blocks so SHA-256 can process it correctly.
It labels itself with the original message, adds a single **"1"** to mark its done, fills the space with **buffer zeros**, and finishes by writing down its **length** at the end!

### Reference
- [*How Does SHA-256 Work?* – Learn Me A Bitcoin (YouTube)](https://www.youtube.com/watch?v=f9EbD6iY9zI)
   Clear visual explanation of padding, including the `1` bit, zero padding, and length field.
- [*Python Generators: A Complete Guide* – Real Python](https://realpython.com/introduction-to-python-generators/)  
  Used to explain how `yield` works and why the function returns one block at a time.
- [*Endianness Explained (Big vs Little Endian)* – GeeksForGeeks](https://www.geeksforgeeks.org/little-and-big-endian-mystery/)  
  Helps explain what "64-bit big-endian length field" means in SHA-256 padding.
- [*ChatGPT (OpenAI)*](https://chatgpt.com/) – Interactive explanations, guidance, markdown used throughout this solution.  


### Example of Message being padded
Lets take the binary number : 
| Step | Description | Result (in bits / hex) |
|:----:|:-------------|:-----------------------|
| 1 | Message | `01100011 01100001 01110100` (`cat`) |
| 2 | Add '1' bit | `...01110100 1` → adds `0x80` |
| 3 | Pad with zeros to reach 448 bits | `00…00` until total = 448 bits |
| 4 | Append 64-bit length (24 in binary) | `00000000 00000000 ... 00011000` |
| 5 | Final padded message = 512 bits (64 bytes) | Ready to split into a single block |

In [22]:
def block_parse(msg: bytes):
    """
    Generator function that pads and splits a message into 512-bit (64-byte) blocks
    according to the SHA-256 rules described in FIPS 180-4.

    Parameters
    ----------
    msg : bytes
        The input message as a bytes object.

    Returns
    ------
    padded : bytes
        The manipulated message after initial padding.
    """
    # Get original message length in bits
    msg_length_bits = len(msg) * 8

    # Add the single '1' bit (0x80 in hex) 
    padded = msg + b'\x80' #10000000

    # Calculate how many zero bytes are needed
    # We need the total length (in bits) ≡ 448 mod 512
    # 64 bytes (the total) minus 8 bytes (the length) = 56 bytes available for data and padding
    while (len(padded) % 64) != 56:
        padded += b'\x00'

    # Add the 64-bit big-endian length
    padded += msg_length_bits.to_bytes(8, 'big')

    # return the padded message
    return padded





In [23]:
#Testing

msg = b"cat"  # 3 bytes = 24 bits
padded = block_parse(msg)

print("Original message (bytes):", msg)
print("Padded message length (bytes):", len(padded))
print("Padded message length (bits):", len(padded) * 8)

# Print hex representation (trimmed)
print("\nPadded message (hex):")
print(padded.hex())

# Print last 8 bytes - the message length in bits
print("\nLast 8 bytes (hex):", padded[-8:].hex())


Original message (bytes): b'cat'
Padded message length (bytes): 64
Padded message length (bits): 512

Padded message (hex):
63617480000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018

Last 8 bytes (hex): 0000000000000018


### Splitting into 512-bit Blocks  

**Split the padded message into 512-bit (64-byte) blocks**.  

Each block will later be processed separately during the hashing stage (Problem 4),  
so this function will be a **generator** that *yields one block at a time* instead of returning the entire padded message.  

- **Yield**: pauses and gives you one piece of data each time you ask for it.

Simply said — we’re about to turn our padded message into small, neat chunks that SHA-256 can process one at a time.


In [24]:
def block_parse(msg: bytes):
    """
    Generator function that pads and splits a message into 512-bit (64-byte) blocks
    according to the SHA-256 rules described in FIPS 180-4.
    """
    # Step 1: Original message length in bits
    msg_length_bits = len(msg) * 8

    # Step 2: Append the single '1' bit (0x80)
    padded = msg + b'\x80'

    # Step 3: Pad with zeros until total length ≡ 56 mod 64 (bytes)
    while (len(padded) % 64) != 56:
        padded += b'\x00'

    # Step 4: Append the 64-bit big-endian length
    padded += msg_length_bits.to_bytes(8, 'big')

    # Step 5: Yield each 512-bit (64-byte) block
    for i in range(0, len(padded), 64): # starts at 0, goes to length of padded, step by 64
        yield padded[i:i + 64] # yields one block at a time


In [25]:
# Testing the generator version of block_parse
test_msg = b"catsarethebestpetstohaveinthewholewideworldandtheyareverycuteandfluffy"  

print("Original message:", test_msg)
print("Original length (bits):", len(test_msg) * 8)
print("=" * 60)

# adds a counter - block1, block2, ...
for index, block in enumerate(block_parse(test_msg), start=1):
    print(f"Block {index}:")
    print("  Length (bytes):", len(block))
    print("  Length (bits):", len(block) * 8)
    print("  Hex preview:", block.hex())
    print("-" * 60)

Original message: b'catsarethebestpetstohaveinthewholewideworldandtheyareverycuteandfluffy'
Original length (bits): 560
Block 1:
  Length (bytes): 64
  Length (bits): 512
  Hex preview: 636174736172657468656265737470657473746f68617665696e74686577686f6c6577696465776f726c64616e64746865796172657665727963757465616e64
------------------------------------------------------------
Block 2:
  Length (bytes): 64
  Length (bits): 512
  Hex preview: 666c7566667980000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000230
------------------------------------------------------------


## <strong>Problem 4 </strong>: Hashes

In this problem we implement the **SHA-256 compression function**, based on   
[Section 6.2.2 of the Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)


This function takes:
- the **current 256-bit hash value**, and  
- the **next 512-bit message block**,  
and produces the **updated hash value**.  

It is the core step that SHA-256 repeats for every message block.

---

### What the Compression Function Does 

1. **Build the Block (W[0..63])**  
   - Take the **first 16 words** straight from the 512-bit block.  
   - Then the algorithm **creates 48 more words** using the σ₀ and σ₁ functions.  
   - This gives us **64 words total**, because SHA-256 does 64 rounds.

2. **Set up the working variables**  
   - Copy the current hash values into:  
     `a, b, c, d, e, f, g, h`  
     These are just temporary variables used during the mixing.

3. **Do 64 mixing rounds**  
   - Each round uses the W words, constants K[t], and the functions Σ₀ (Sigma0), Σ₁ (Sigma1), Ch, Maj (From Problem 1)
   - The variables `a..h` change a little bit each round.

4. **Produce the new hash value**  
   - After the 64 rounds finish, the final `a..h` values are **added back** to the original hash.  
   - This becomes the **updated hash**, ready for the next block (if there is one).

---

### Simply Said


The compression function is the step that actually performs the hashing.  

It takes one padded 512-bit block of the message and the current hash value, expands the block into 64 words, runs 64 rounds of calculations, and then combines the result with the current hash. 

The output becomes the new hash value, which will be used for the next block or returned as the final SHA-256 hash if this is the only block.


---

### Example 

| Stage | Output (simplified) |
|-------|----------------------|
| Message | `"cat"` |
| After padding | 1 × 512-bit block `63617480 … 00000018` |
| W[0..15] | W[0] = 63617480<br>W[1] = 00000000<br>W[2] = 00000000<br>…<br>W[14] = 00000000<br>W[15] = 00000018 |
| Rounds 0–63 | State updated |
| Final hash | `d077f244de...` |

(That is the real SHA-256 of `"cat"`.)

---

### References Used

- [Section 6.2.2 of the Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- Functions from Problem 1 (Ch, Maj, Σ₀, Σ₁, σ₀, σ₁)  
- Constants from Problem 2  
- Padding & block parsing from Problem 3  
- [*ChatGPT (OpenAI)*](https://chatgpt.com/) – Interactive explanations, guidance, markdown used throughout this solution.


#### Step 1: Convert a 512-bit block into W[0..15]

According to the Secure Hash Standard (FIPS 180-4, §5.2.1 and §6.2.2), a 512-bit message block is treated as **sixteen 32-bit words** in big-endian order. This step takes a padded 512-bit block (64 bytes) and splits it into the first sixteen words of the message schedule:

- Each word is 4 bytes (32 bits).
- We read the block 4 bytes at a time.
- Each group of 4 bytes is converted to a 32-bit unsigned integer using big-endian byte order.
- The result is an array `W[0..15]` that we will later expand to `W[0..63]`.


In [26]:
def initial_words_from_block(block: bytes) -> np.ndarray:
    """
    Convert a 512-bit (64-byte) message block into the first
    sixteen 32-bit words W[0..15] for SHA-256.

    Each word is read in big-endian order, as specified in
    FIPS 180-4 (see §5.2.1 and §6.2.2):
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

    Parameters
    ----------
    block : bytes
        A single 512-bit message block (exactly 64 bytes) produced
        after padding and parsing (See Problem 3).

    Returns
    -------
    numpy.ndarray
        A NumPy array of shape (16,) with dtype uint32, containing
        the words W[0]..W[15].
    """
    # Error Handling: SHA-256 compression always operates on 64-byte blocks.
    if len(block) != 64:
        raise ValueError(f"Block must be exactly 64 bytes (got {len(block)})")

    # Allocate an array of 16 unsigned 32-bit integers for W[0..15].
    W = np.zeros(16, dtype=np.uint32)

    # Process the block 4 bytes at a time (big-endian) to build W[0..15].
    for i in range(16):
        # Take 4 bytes: block[i*4 : (i+1)*4]
        word_bytes = block[i * 4 : (i + 1) * 4]

        # Convert those 4 bytes into a 32-bit integer (big-endian).
        W[i] = int.from_bytes(word_bytes, byteorder="big")

    return W


In [27]:
# Example test for Step 1: using a known 512-bit block in hex.
# e.g. "63617480....00000018" (128 hex characters = 64 bytes).

blocks = list(block_parse(b"cat")) # Generate blocks for "cat" from Problem 3 function
block = blocks[0]   

print(f"Block length (bytes): {len(block)}")  # should be 64

# Convert block into W[0..15]
W_0_15 = initial_words_from_block(block)

print("\nW[0..15] as hex:")
for i, w in enumerate(W_0_15):
    print(f"W[{i:2}] = {w:08x}")



Block length (bytes): 64

W[0..15] as hex:
W[ 0] = 63617480
W[ 1] = 00000000
W[ 2] = 00000000
W[ 3] = 00000000
W[ 4] = 00000000
W[ 5] = 00000000
W[ 6] = 00000000
W[ 7] = 00000000
W[ 8] = 00000000
W[ 9] = 00000000
W[10] = 00000000
W[11] = 00000000
W[12] = 00000000
W[13] = 00000000
W[14] = 00000000
W[15] = 00000018


### Step 2 — Extending the Message Schedule (W[16..63])

After computing the first sixteen words `W[0..15]`, SHA-256 expands them into a full **64-word message schedule**.  

---

### What This Step Is Doing

A single SHA-256 block uses **64 rounds**, but the message block only provides **16 words**.  
To produce the remaining **48 words**, the algorithm uses the “small sigma” functions σ₀ and σ₁ along with earlier values of W (W = The original message).

For each `t` from **16 to 63**:

$$
    W_t = \sigma_1(W_{t-2}) + W_{t-7} + \sigma_0(W_{t-15}) + W_{t-16} \pmod{2^{32}}
$$

Where:

- **σ₀(x)** = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)  
- **σ₁(x)** = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)

These functions were implemented in Problem 1.

This step spreads (diffuses) information from the original block so that every bit influences many later rounds of the computation.

---

### Simply Said

We start with **16 words**, but SHA-256 needs **64**.  
So it creates 48 new ones by rotating, shifting, and combining earlier W values.  
This ensures the message is fully mixed before the compression rounds begin.

---

### Example (conceptual)

```
W[16] = σ1(W[14]) + W[9]  + σ0(W[1])  + W[0]
W[17] = σ1(W[15]) + W[10] + σ0(W[2])  + W[1]
...
W[63] = σ1(W[61]) + W[56] + σ0(W[48]) + W[47]
```

---

### Alternative Ways to Implement Step 2

Below are two other valid implementations of the W-extension step and why the notebook does not use them:

| Version | Example Code | Why Not Used |
|--------|--------------|--------------|
| **1. Pre-binding functions** | `s0 = sigma0`<br>`s1 = sigma1`<br>`W[t] = s1(W[t-2]) + ...` | Slightly cleaner but less clear for readers comparing to FIPS pseudocode. Minimal benefit. |
| **2. NumPy vectorised slicing** | `W[16:] = σ1(W[14:62]) + ...` | Hard to read, easy to misindex, σ functions do not vectorise cleanly. Too complex for simple demonstration |

**Conclusion:**  
The simple loop version is used because it is clearest, easiest to mark, and most faithful to the FIPS 180-4 structure.


### References Used

- [Section 6.2.2 of the Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) 
- σ₀ and σ₁ functions from Problem 1  
- W[0..15] from Step 1

---


In [28]:
def extend_message_schedule(W: np.ndarray) -> np.ndarray:
    """
    Extend the message schedule from W[0..15] to W[0..63],
    following FIPS 180-4, Section 6.2.2.

    Parameters
    ----------
    W : np.ndarray
        Array of 64 uint32 entries, where W[0..15] are already filled.

    Returns
    -------
    np.ndarray
        The same array W, with W[16..63] now computed.
    """
    for t in range(16, 64):
        W[t] = (
            sigma1(W[t - 2])
            + W[t - 7]
            + sigma0(W[t - 15])
            + W[t - 16]
        )

    return W


In [29]:
# --- Testing Step 2: Extend W[0..15] to W[0..63] ---

# Generate a block from "cat"
blocks = list(block_parse(b"cat"))
block = blocks[0]

# Compute W[0..15]
W_0_15 = initial_words_from_block(block)

# Make a full W array (64 words), placing W_0_15 and leaving the rest zero
W = np.zeros(64, dtype=np.uint32)
W[:16] = W_0_15

# Extend the message schedule
extended_W = extend_message_schedule(W.copy())

# Print a preview
print("W[0..19] (hex):")
for i in range(20):
    print(f"W[{i:2}] = {extended_W[i]:08x}")

# Basic structural tests
print("\nChecks:")
print("Length =", len(extended_W))                         # should be 64
print("dtype  =", extended_W.dtype)                        # should be uint32
print("W[16]  =", hex(int(extended_W[16])))                # should NOT be 0
print("W[17]  =", hex(int(extended_W[17])))                # should NOT be 0

# Repeat extension to confirm same result
W2 = np.zeros(64, dtype=np.uint32)
W2[:16] = W_0_15
extended_W2 = extend_message_schedule(W2.copy())

print("\nSame Hash? =", np.array_equal(extended_W, extended_W2))


W[0..19] (hex):
W[ 0] = 63617480
W[ 1] = 00000000
W[ 2] = 00000000
W[ 3] = 00000000
W[ 4] = 00000000
W[ 5] = 00000000
W[ 6] = 00000000
W[ 7] = 00000000
W[ 8] = 00000000
W[ 9] = 00000000
W[10] = 00000000
W[11] = 00000000
W[12] = 00000000
W[13] = 00000000
W[14] = 00000000
W[15] = 00000018
W[16] = 63617480
W[17] = 000f0000
W[18] = 94c8e581
W[19] = 600003c6

Checks:
Length = 64
dtype  = uint32
W[16]  = 0x63617480
W[17]  = 0xf0000

Same Hash? = True


### Step 3 — Initialising the Working Variables (a–h)

Before the 64 compression rounds begin, SHA-256 copies the **current hash value** into eight working variables:

$$
a, b, c, d, e, f, g, h
$$

For the **very first block**, this “current hash value” is simply the **fixed initial hash constants** H₀–H₇ defined in FIPS 180-4.  
These act as the starting state for the algorithm.

The working variables are then updated throughout the 64 rounds in Step 4.

---

### What This Step Is Doing

- Takes the current hash  
  – first block → the fixed initial constants  
  – later blocks → the previous block’s hash  
- Copies those 8 words into the variables `a..h`  
- These variables will be modified during the 64-round compression loop

Simply put:  
- Step 3 loads the starting state into a–h before mixing begins.

---

### Different Ways to Implement Step 3

| Method | Example | Notes |
|--------|---------|-------|
| **Slice unpacking (used here)** | `a, b, c, d, e, f, g, h = current[:8]` | Clean, short, and works identically to direct unpacking. |
| **Direct unpacking** | `a, b, …, h = current` | Very clear; matches most pseudocode. |
| **Manual indexing** | `a = current[0]`, … | Explicit but unnecessarily long. |

We use **slice unpacking** because it is short, neat, and fits cleanly in the code.

---


In [30]:
def initialise_working_variables(current: np.ndarray):
    """
    Load the current hash value H(i-1) into the eight
    working variables a..h used during the 64 rounds.

    Parameters
    ----------
    current : np.ndarray
        Array of 8 uint32 values representing the current hash.

    Returns
    -------
    tuple
        Tuple (a, b, c, d, e, f, g, h) of uint32 values.
    """
    a, b, c, d, e, f, g, h = current[:8]  # Using a NumPy Slice

    return a, b, c, d, e, f, g, h


#### Quick Step - Set up SHA-256 Initial Hash Values (H0..H7)

Before the compression function can begin, SHA-256 starts with a fixed 256-bit initial state.  
These eight 32-bit constants are defined in FIPS 180-4 and are the same for every SHA-256 implementation.  
They act as the starting point for hashing the first message block.


In [31]:
# SHA-256 Initial Hash Values (H0..H7)
# These are the fixed 256-bit starting state defined in FIPS 180-4.
# Every SHA-256 implementation in the world begins hashing with these values.
# They are derived from the fractional parts of the square roots of the first 8 primes.

H0 = np.array([
    0x6a09e667,
    0xbb67ae85,
    0x3c6ef372,
    0xa54ff53a,
    0x510e527f,
    0x9b05688c,
    0x1f83d9ab,
    0x5be0cd19
], dtype=np.uint32)


In [32]:
# Test Step 3: Initialising working variables

# Example current hash (use SHA-256 initial constants from Problem 2)
current = H0.copy()   # assuming you stored your initial hash constants as H0

# Initialise a..h
a, b, c, d, e, f, g, h = initialise_working_variables(current)

print("Working variables loaded:")
print(f"a = {a:08x}")
print(f"b = {b:08x}")
print(f"c = {c:08x}")
print(f"d = {d:08x}")
print(f"e = {e:08x}")
print(f"f = {f:08x}")
print(f"g = {g:08x}")
print(f"h = {h:08x}")

print("\nMatch with current hash:", 
      np.array_equal(current, np.array([a,b,c,d,e,f,g,h], dtype=np.uint32)))


Working variables loaded:
a = 6a09e667
b = bb67ae85
c = 3c6ef372
d = a54ff53a
e = 510e527f
f = 9b05688c
g = 1f83d9ab
h = 5be0cd19

Match with current hash: True


### Step 4 — Performing the 64 Compression Rounds

With the working variables `a..h` loaded and the full message schedule `W[0..63]` prepared, SHA-256 now performs **64 rounds** of mixing.  
Each round updates the working variables using the functions Σ₀, Σ₁, Ch, Maj, the constant K[t], and the message word W[t].

This is the core of the compression function.

---

### What This Step Is Doing

SHA-256 performs **64 rounds**, and in each round it computes two temporary values and then updates the eight working variables.

#### Temporary Values (Per Round)

| Value | Formula |
|-------|---------|
| **T1** | h + Σ₁(e) + Ch(e, f, g) + K[t] + W[t] |
| **T2** | Σ₀(a) + Maj(a, b, c) |



#### Working Variable Updates (Per Round)

| New Variable | Becomes | Notes |
|--------------|---------|--------|
| **h** | g | shift down |
| **g** | f | shift down |
| **f** | e | shift down |
| **e** | d + T1 | only `e` receives T1 |
| **d** | c | shift down |
| **c** | b | shift down |
| **b** | a | shift down |
| **a** | T1 + T2 | main update combining message + state |


Because everything is stored as `uint32`, all additions automatically wrap around modulo 2³², exactly as required by FIPS 180-4.

---


### Simply Said

The message schedule W and constants K are fed into the algorithm round-by-round.  
Each round “pushes” the values a–h forward and changes them slightly.  
After 64 rounds, the working variables contain the fully mixed state for this block.

---

### References Used

- [Secure Hash Standard (FIPS 180-4), Section 6.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- Problem One - Binary Words and Operations

---


In [33]:
def perform_rounds(a, b, c, d, e, f, g, h, W):
    """
    Perform the 64 SHA-256 compression rounds.

    Parameters
    ----------
    a, b, c, d, e, f, g, h : uint32
        The working variables initialised in Step 3.
    W : np.ndarray
        The message schedule W[0..63].

    Returns
    -------
    tuple
        Updated working variables (a, b, c, d, e, f, g, h)
        after completing all 64 rounds.
    """

    K = constants  # SHA-256 round constants from Problem 2
    
    for t in range(64):
        T1 = (
            h
            + Sigma1(e)
            + choose(e, f, g)
            + K[t] 
            + W[t]
        )

        T2 = Sigma0(a) + Maj(a, b, c)

        # Update working variables
        h = g
        g = f
        f = e
        e = d + T1
        d = c
        c = b
        b = a
        a = T1 + T2

    return a, b, c, d, e, f, g, h


### Step 5 — Producing the Updated Hash Value

After completing the 64 compression rounds, the working variables  
`a, b, c, d, e, f, g, h` contain the fully mixed state for this block.

As per SHA-256, we will now combines these values with the *previous* hash value H(i−1)  
to produce the *new* hash value H(i).

---

### What This Step Is Doing

After 64 rounds, we compute the updated hash by adding each working variable  
back to its corresponding previous hash value (modulo 2³²):

<table>
<tr>
<td>

$$
\begin{aligned}
H_0^{(i)} &= H_0^{(i-1)} + a \\
H_1^{(i)} &= H_1^{(i-1)} + b \\
H_2^{(i)} &= H_2^{(i-1)} + c \\
H_3^{(i)} &= H_3^{(i-1)} + d \\
H_4^{(i)} &= H_4^{(i-1)} + e \\
H_5^{(i)} &= H_5^{(i-1)} + f \\
H_6^{(i)} &= H_6^{(i-1)} + g \\
H_7^{(i)} &= H_7^{(i-1)} + h \\
\end{aligned}
$$

</td>
<td>

| Part | Interpretation |
|------|----------------|
| **Hₖ⁽ⁱ⁻¹⁾** | The old hash (what we started this block with). |
| **Hₖ⁽ⁱ⁾** | The new hash (old hash + this block's contribution). |
| **a..h** | The mixed results from the 64 rounds. |

</td>
</tr>
</table>

Because all values are stored as `uint32`, these additions automatically wrap  
around modulo 2³² as required by the standard.

---

### Simply Said

Step 5 "closes the loop":

We just take the final mixed values (a–h) and add them back to the old hash.  
That's it — the hash has now been updated.  

If there were more blocks, we'd keep going.  

---

### References Used

- [FIPS 180-4, Section 6.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- [Online SHA256 Calculator](https://emn178.github.io/online-tools/sha256.html)

---


In [34]:
def update_hash_value(current_hash, a, b, c, d, e, f, g, h):
    """
    Update the SHA-256 hash value after completing the 64 rounds.

    Each working variable (a..h) is added to the corresponding word in the
    previous hash value H(i-1), producing the new hash H(i).

    Parameters
    ----------
    current_hash : np.ndarray
        The previous hash value H(i-1) as an array of 8 uint32 values.
    a, b, c, d, e, f, g, h : uint32
        The working variables after 64 rounds.

    Returns
    -------
    np.ndarray
        The updated hash value H(i), as 8 uint32 values.
    """

    return np.array([
        current_hash[0] + a,
        current_hash[1] + b,
        current_hash[2] + c,
        current_hash[3] + d,
        current_hash[4] + e,
        current_hash[5] + f,
        current_hash[6] + g,
        current_hash[7] + h
    ], dtype=np.uint32)


---

### Complete Hash Function

As per the assignment requirements, the function `hash(current, block)` combines all the steps above to calculate the next hash value given the current hash value and the next message block, following Section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

This function integrates:
- **Step 1**: Converting the block into W[0..15]
- **Step 2**: Extending the message schedule to W[0..63]
- **Step 3**: Initialising working variables from the current hash
- **Step 4**: Performing the 64 compression rounds
- **Step 5**: Producing the updated hash value

---

In [35]:
def hash(current, block):
    """
    Calculate the next hash value given the current hash value and 
    the next message block according to Section 6.2.2 SHA-256 Hash 
    Computation (page 22) of the Secure Hash Standard (FIPS 180-4).

    This function implements the complete SHA-256 compression function,
    combining all five steps:
    1. Prepare the message schedule W[0..15] from the block
    2. Extend W to W[0..63]
    3. Initialize working variables a..h from current hash
    4. Perform 64 rounds of compression
    5. Compute and return the updated hash value

    Parameters
    ----------
    current : np.ndarray
        The current hash value H(i-1) as an array of 8 uint32 values.
    block : bytes
        A 512-bit (64-byte) message block.

    Returns
    -------
    np.ndarray
        The updated hash value H(i) as an array of 8 uint32 values.

    References
    ----------
    FIPS 180-4, Section 6.2.2: SHA-256 Hash Computation
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    """
    
    # Step 1: Prepare message schedule W[0..15]
    W = np.zeros(64, dtype=np.uint32)
    W[:16] = initial_words_from_block(block)
    
    # Step 2: Extend message schedule to W[0..63]
    extend_message_schedule(W)
    
    # Step 3: Initialize working variables
    a, b, c, d, e, f, g, h = initialise_working_variables(current)
    
    # Step 4: Perform 64 compression rounds
    a, b, c, d, e, f, g, h = perform_rounds(a, b, c, d, e, f, g, h, W)
    
    # Step 5: Update and return the hash value
    return update_hash_value(current, a, b, c, d, e, f, g, h)

In [36]:
# ---- FINAL TEST: Using hash(current, block) function ----

# Get padded blocks from "cat"
blocks = list(block_parse(b"cat"))
block = blocks[0]

# Calculate hash using the hash(current, block) function
# Start with the initial hash value H0
H1 = hash(H0, block)

# Convert to final digest
final_digest = ''.join(f'{x:08x}' for x in H1)

print("Final SHA-256 digest for 'cat':")
print(final_digest)

# ONLINE SHA256 CALCULATOR RESULT
online_result = '77af778b51abd4a3c51c5ddd97204a9c3ae614ebccb75a606c3b6865aed6744e'

if final_digest == online_result:
    print("\nHash matches online SHA-256 calculator")
else:
    print(f"\nMismatch")
    print(f"Expected: {online_result}")
    print(f"Got:      {final_digest}")

Final SHA-256 digest for 'cat':
77af778b51abd4a3c51c5ddd97204a9c3ae614ebccb75a606c3b6865aed6744e

Hash matches online SHA-256 calculator


## <strong>Problem 5 </strong>: Passwords

SHA-256 takes a password and turns it into a fixed 256-bit hash. It’s designed to be fast and reliable for checking data, but that also makes it a bad choice for storing passwords. Because it’s so fast, an attacker can easily hash huge lists of common passwords and simply compare the results to the hash they’re trying to crack.

They’re not “reversing” the hash, they’re just finding a password that produces the same output. And since weak passwords like the ones in this problem appear in basically every common wordlist and online hash database, it’s extremely quick to identify them. So in this case, all I needed to do was match the given hashes to known SHA-256 hashes of common passwords.

---

## 2. The Passwords
The passwords corresponding to the given SHA-256 hashes were recovered by testing likely candidate passwords and comparing their SHA-256 hashes to the provided values. This approach relies on the fact that SHA-256 is deterministic: the same input will always produce the same hash.

The recovered passwords are shown below.

| Hash | Recovered Password |
|------|--------------------|
| 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8 | **password** |
| 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34 | **cheese** |
| b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342 | **P@ssw0rd** |

### How the passwords were found

>References:  
>(CrackStation – Online Hash Lookup)[https://crackstation.net]  
>(Hashes.com – Public Hash Database)[https://hashes.com]  
>(MD5Hashing – SHA-256 Lookup Tool)[https://md5hashing.net]


- **`password`**  
  I identified this password by looking up the hash using an online SHA-256 hash lookup tool that checks against lists of common passwords. The hash was found to correspond to the plaintext value “password”, which was then confirmed by verifying the hash.

- **`cheese`**  
  I recovered this password using an online hash lookup tool that relies on precomputed hash datasets, similar to rainbow tables. Because the hash was unsalted, it could be directly matched to the plaintext word “cheese” stored in the lookup database.

- **`P@ssw0rd`**  
  I found this password using a hash lookup tool that includes common password variations and mutations. These tools test predictable transformations such as symbol substitutions and changes in character case, which led to a match for the plaintext value “P@ssw0rd”.

In each case, I did not reverse the SHA-256 hash function. Instead, I identified candidate inputs and verified them by comparing their computed hashes with the provided hash values.


---

## 3. Approaches to Cracking SHA-256 Password Hashes

There are several common approaches attackers can use to recover passwords from SHA-256 hashes. These techniques do not reverse the hash function itself, but instead try likely inputs and compare their hashes to the target value. 

### 3.1 Dictionary Attack
A dictionary attack involves hashing words from a predefined list, such as common passwords or English words, and checking for a match. This is one of the most effective techniques against weak passwords and was the main approach used to recover the passwords in this problem.

Reference:  
(What are Dictionary attacks – Jump Cloud)[https://jumpcloud.com/it-index/what-are-dictionary-attacks]

### 3.2 Brute-Force Attack
A brute-force attack tries every possible combination of characters until a matching hash is found. While this approach is guaranteed to succeed in theory, it quickly becomes impractical as password length increases due to the exponential number of combinations.

Reference:  
(Brute force attacks explained – Cloudflare)[https://www.cloudflare.com/learning/security/threats/brute-force-attack/]

### 3.3 Rainbow Tables
Rainbow tables are large, precomputed tables that store plaintext passwords alongside their corresponding hashes. They allow for extremely fast lookups but require significant storage space and are ineffective when salts are used. Rainbow tables are mainly useful against unsalted hash databases.

Reference:  
(Understanding Rainbow tables – GeeksforGeeks)[https://www.geeksforgeeks.org/ethical-hacking/understanding-rainbow-table-attack/]

### 3.4 Hash Lookup Databases
Hash lookup databases store collections of previously cracked hashes. If a given hash exists in one of these databases, the original password can be retrieved almost instantly. This method works well for very common passwords that appear frequently in data breaches.

Reference:  
(md5 Hashing – Online Hash Lookup)[https://md5hashing.net/hash/sha256]

### 3.5 Mask/Hybrid Attacks
Mask or hybrid attacks combine dictionary attacks with common patterns or rules, such as capitalisation, number suffixes, or symbol substitutions. This approach is effective against passwords that follow predictable human patterns, such as replacing letters with numbers or symbols.

Reference:  
(Hashcat rule-based attacks explained)[https://hashcat.net/wiki/doku.php?id=rule_based_attack]


### 3.6 Machine Learning / Probabilistic Guessing Models
More advanced techniques use machine learning to generate likely password guesses based on real leaked password datasets. These models prioritise guesses that resemble real human password choices, improving efficiency compared to traditional brute-force methods.

Reference:  
(PassTCN-PPLL: A Password Guessing Model Based on Probability Label Learning and Temporal Convolutional Neural Network - Junbin Ye)[https://pmc.ncbi.nlm.nih.gov/articles/PMC9459998/]

### 3.7 Markov Chain Attacks

Markov chain attacks generate password guesses based on the statistical likelihood of character sequences occurring in real passwords. Instead of treating each character independently, the attack models how likely one character is to follow another, based on previously observed password data.

This allows the attacker to prioritise guesses that resemble human-created passwords, making the attack more efficient than pure brute force. While more advanced than a simple dictionary attack, Markov-based approaches are still practical and are implemented in real-world password cracking tools.

For the passwords in this problem, simpler dictionary-based methods were already sufficient, so a Markov chain attack would not provide a significant advantage. However, it is effective against longer, more complex passwords that still follow human patterns.

Reference:  
(Using Markov Models to Crack Passwords - R. P. van Heerden and J.S. Vorster)[https://www.researchgate.net/profile/Renier-Van-Heerden/publication/269098736_Using_Markov_Models_to_Crack_Passwords/links/5480134c0cf250f1edbfab8a/Using-Markov-Models-to-Crack-Passwords.pdf]

---

## 4. Comparison of Approaches

For the passwords in this task, rainbow tables and dictionary attacks are the most effective and practical approaches. Rainbow tables allow instant identification of well-known passwords through precomputed hash lookups, while dictionary attacks work by hashing likely candidates and comparing them directly. Since all of the recovered passwords were weak or highly predictable, these methods were more than sufficient.

More advanced techniques such as brute-force attacks or GPU-accelerated cracking are unnecessary in this case, as they are designed for exploring much larger search spaces. Approaches like Markov chain attacks are more intuitive and human-like, as they prioritise guesses that resemble real password patterns, but even these are excessive for passwords that already appear in common wordlists or lookup databases.

--- 

## 5. Improving Password Hashing

The passwords in this task were recovered because they were hashed using SHA-256 without additional protections. While SHA-256 is a secure cryptographic hash function, it is not well designed for password storage. The following improvements would prevent the types of attacks used to recover the passwords in this problem.

### 5.1 Salting Passwords

Passwords should always be salted before hashing. A salt is a random value added to the password prior to hashing, ensuring that identical passwords do not result in identical hashes. This prevents the use of rainbow tables and hash lookup databases, as precomputed hashes will no longer match stored values.

### 5.2 Key Stretching and Work Factors

Key stretching involves hashing a password multiple times using a configurable work factor. Increasing the number of iterations raises the computational cost of each guess, making large-scale password guessing attacks far more time-consuming while remaining practical for legitimate authentication.

### 5.3 Adding a Secret Pepper

A pepper is an additional secret value combined with the password during hashing, similar to a salt, but stored separately from the password database. Even if an attacker gains access to the hashed passwords, the absence of the pepper prevents direct verification of guessed passwords.

### 5.4 Memory-Hard Hashing Techniques

More advanced memory-hard hashing techniques, such as Balloon hashing, further improve resistance to offline attacks. These approaches require both significant memory and CPU resources, making attacks using GPUs or specialised hardware much less effective. Memory-hard functions directly address the scalability advantages attackers gain from parallel hardware.

### 5.5 Conclusion
In practice, the most effective defence against password cracking is not a single technique, but a combination of several approaches. Using salted, slow, password-specific hashing algorithms alongside key stretching and memory-hard constructions would significantly reduce the feasibility of dictionary, rainbow table, and rule-based attacks. Adding a secret pepper provides an additional layer of protection even if the password database is compromised. However, these improvements come with increased computational cost and implementation complexity, and some newer techniques require further evaluation before widespread adoption. As a result, real-world systems must balance security, performance, and maturity when choosing a password hashing strategy.


>References
>(Password salting explained – Paubox)[https://www.paubox.com/blog/what-is-password-salting]  
>(Password storage best practices – OWASP)[https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html]  
>(Argon2 overview – Wikipedia)[https://en.wikipedia.org/wiki/Argon2]  
>(Pepper in cryptography – Wikipedia)[https://en.wikipedia.org/wiki/Pepper_%28cryptography%29]  
>(Balloon hashing – Wikipedia)[https://en.wikipedia.org/wiki/Balloon_hashing]

---

